# Introduction to Interact.jl

In [1]:
using Reactive, Interact

Interact.jl provides interactive widgets for IJulia. Interaction relies on [Reactive.jl](http://julialang.org/Reactive.jl/) reactive programming package. Reactive provides the type `Signal` which represent time-varying values. For example, a Slider widget can be turned into a "signal of numbers". Execute the following two cells, and then move the slider. You will see that the value of `signal(slider)` changes accordingly.

In [2]:
s = slider(0:.1:1,label="Slider X:")

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=1),"Slider X:",0.5,0.0:0.1:1.0,"horizontal",true,".3f",true)

In [3]:
signal(s)

0.5

Let us now inspect the types of these entities.

In [4]:
display(typeof(s));
isa(s, Widget)

Interact.Slider{Float64}

true

In [5]:
display(typeof(signal(s)));
isa(signal(s), Signal{Float64})

Reactive.Signal{Float64}

true

You can have many instances of the same widget in a notebook, and they stay in sync:

In [6]:
s

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=2),"Slider X:",0.5,0.0:0.1:1.0,"horizontal",true,".3f",true)

## Using Widget Signals

A slider is useless if you cannot do more with it than just watch its value. Thankfully we can transform one signal into another, which means we can transform the signal of values that the slider takes into, say a signal of it's squares:

In [7]:
xsquared = map(x -> x*x, signal(s))

0.25

Go ahead and vary the slider to see this in action.

You can transform a signal into pretty much anything else. Let's use the Color package to produce different saturations of red:

In [8]:
using Colors
map(x -> RGB(x, 0.5, 0.5), signal(s))

You can of course use several inputs as arguments to `map` enabling you to combine many signals. Let's create a full color-picker.

In [9]:
r = slider(0:0.01:1, label="R")
g = slider(0:0.01:1, label="G")
b = slider(0:0.01:1, label="B")
map(display, [r,g,b]);

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=1),"R",0.5,0.0:0.01:1.0,"horizontal",true,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=1),"G",0.5,0.0:0.01:1.0,"horizontal",true,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=1),"B",0.5,0.0:0.01:1.0,"horizontal",true,".3f",true)

In [10]:
color = map((x, y, z) -> RGB(x, y, z), signal(r), signal(g), signal(b))

We can use the HTML widget to write some text you can change the color of.

In [11]:
map(color -> HTML(string("<div style='color:#", hex(color), "'>Hello, World!</div>")), signal(color))

HTML{String}("<div style='color:#808080'>Hello, World!</div>")

## The @manipulate Macro

The `@manipulate` macro lets you play with any expression using widgets. We could have, for example, used `@manipulate` to make a color picker along with our HTML output in one line of code:

In [12]:
@manipulate for r = 0:.05:1, g = 0:.05:1, b = 0:.05:1
    HTML(string("<div style='color:#", hex(RGB(r,g,b)), "'>Color me</div>"))
end

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.5, nactions=1),"r",0.5,"0.5",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.05"=>0.05,"0.1"=>0.1,"0.15"=>0.15,"0.2"=>0.2,"0.25"=>0.25,"0.3"=>0.3,"0.35"=>0.35,"0.4"=>0.4,"0.45"=>0.45…),Dict(1.0=>"1.0",0.95=>"0.95",0.3=>"0.3",0.45=>"0.45",0.25=>"0.25",0.35=>"0.35",0.7=>"0.7",0.0=>"0.0",0.85=>"0.85",0.15=>"0.15"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.5, nactions=1),"g",0.5,"0.5",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.05"=>0.05,"0.1"=>0.1,"0.15"=>0.15,"0.2"=>0.2,"0.25"=>0.25,"0.3"=>0.3,"0.35"=>0.35,"0.4"=>0.4,"0.45"=>0.45…),Dict(1.0=>"1.0",0.95=>"0.95",0.3=>"0.3",0.45=>"0.45",0.25=>"0.25",0.35=>"0.35",0.7=>"0.7",0.0=>"0.0",0.85=>"0.85",0.15=>"0.15"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.5, nactions=1),"b",0.5,"0.5",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.05"=>0.05,"0.1"=>0.1,"0.15"=>0.15,"0.2"=>0.2,"0.25"=>0.25,"0.3"=>0.3,"0.35"=>0.35,"0.4"=>0.4,"0.45"=>0.45…),Dict(1.0=>"1.0",0.95=>"0.95",0.3=>"0.3",0.45=>"0.45",0.25=>"0.25",0.35=>"0.35",0.7=>"0.7",0.0=>"0.0",0.85=>"0.85",0.15=>"0.15"…)),Any[],Any[],true,"horizontal")

HTML{String}("<div style='color:#808080'>Color me</div>")

## Signal of Widgets

You can in fact create signal of other widgets to update them reactively. We have seen one case with `HTML` above. Let us now create a signal of Sliders:

In [13]:
x = slider(0:.1:2pi, label="x")
s = map(a -> slider(-1:.05:1, value=sin(2a), label="sin(2x)"), signal(x))
c = map(a -> slider(-1:.05:1, value=cos(2a), label="cos(2x)"), signal(x))
map(display, [x,s,c]);

Interact.Slider{Float64}(Signal{Float64}(3.1, nactions=3),"x",3.1,0.0:0.1:6.2,"horizontal",true,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(-0.0830894028174964, nactions=1),"sin(2x)",-0.0830894028174964,-1.0:0.05:1.0,"horizontal",true,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.9965420970232175, nactions=1),"cos(2x)",0.9965420970232175,-1.0:0.05:1.0,"horizontal",true,".3f",true)

Now vary the x slider to see sin(2x) and cos(2x) get set to their appropriate values.

But in the above case, you cannot also use sin(2x) and cos(2x) sliders as input values (without doing tricky stuff beyond the scope of this introduction). To do this more simply, we will create a separate Input signal and pass it as an argument to map, to be the signal that the widget updates. Example:

In [14]:
fx = Signal(0.0) # A float input

0.0

In [15]:
x = slider(0:.1:2pi, label="x")
y = map(v -> slider(-1:.05:1, value=sin(v), signal=fx, label="f(x)"), signal(x))
map(display, (x,y,fx));

Interact.Slider{Float64}(Signal{Float64}(3.1, nactions=2),"x",3.1,0.0:0.1:6.2,"horizontal",true,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.04158066243329049, nactions=2),"f(x)",0.04158066243329049,-1.0:0.05:1.0,"horizontal",true,".3f",true)

0.04158066243329049

f(x) will update as x changes. But if the user slides f(x) then the `fx` signal takes the value chosen by the user.